## EfficientNet

In [1]:
#!c1.4
import os
import zipfile

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


from data_prepare.dataset_tools import extract_zip_with_cleanup, prepare_and_save_data
from data_prepare.f1score import F1Score

import pandas as pd
import matplotlib.pyplot as plt

from data_prepare.dataset_tools import extract_zip_with_cleanup, prepare_and_save_data
from data_prepare.plots import plot_history, confusion_matrix_plot, roc_plot, precision_recall_plot
from data_prepare.models import ModelType, DataLoader, NetFineTunedModel, FeatureExtractorModel

In [2]:
IMAGE_ARCHIVE_PATH="data/celeb/v1/"
MODELS_DIR = "models"
RESULTS_DIR = "results/net"
MODEL_TYPE=ModelType.EFFICIENTNET
DATASET_OUTPUT_DIR='data/dataset'
print('Done')

Done


## Pure model

In [32]:
image_archive_path = "data/celeb/v1/"
fake_images_path, real_images_path = extract_zip_with_cleanup(image_archive_path)

In [33]:
train_dir, val_dir, test_dir = prepare_and_save_data(real_images_path, fake_images_path, output_dir="data/dataset/net", target_size=(224,224))

In [34]:
train_gen, val_gen, test_gen = create_data_generators(train_dir, val_dir, test_dir, target_size=(224,224))

Found 9075 images belonging to 2 classes.
Found 1944 images belonging to 2 classes.
Found 1946 images belonging to 2 classes.

Class indices: {'fake': 0, 'real': 1}
Train samples: 9075
Val samples: 1944
Test samples: 1946


### Обучение модели

In [9]:
#!gt4.1
test_loss, test_precision, test_recall, test_auc = trained_model.evaluate(test_gen)
print(f"\nTest Accuracy: {test_auc:.4f}")

2025-04-29 21:15:58.167842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


61/61 [==============================] - 14s 233ms/step - loss: 0.4537 - accuracy: 0.8356 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5196


ValueError: too many values to unpack (expected 4)

## Finetune model

### Подготовка данных

In [4]:
data_loader = DataLoader(IMAGE_ARCHIVE_PATH, MODEL_TYPE)
train_gen, val_gen, test_gen = data_loader.load_genrators(DATASET_OUTPUT_DIR)
print("Done")

data/celeb/v1/fake data/celeb/v1/real
data/dataset/net/train data/dataset/net/val data/dataset/net/test
Found 112 images belonging to 2 classes.
Found 1944 images belonging to 2 classes.
Found 1946 images belonging to 2 classes.

Class indices: {'fake': 0, 'real': 1}
Train samples: 112
Validation samples: 1944
Test samples: 1946
Done


In [5]:
finetune = NetFineTunedModel(MODELS_DIR)
finetune_model, finetune_history = finetune.fit(train_gen, val_gen, 3, 7)

Epoch 1/3


/Users/aleksandr/opt/anaconda3/envs/deepfakedetection/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 848ms/step - auc: 0.5730 - loss: 1.0176 - precision: 0.1170 - recall: 0.7960

4/4 ━━━━━━━━━━━━━━━━━━━━ 25s 5s/step - auc: 0.5852 - loss: 0.9727 - precision: 0.1222 - recall: 0.7641 - val_auc: 0.5474 - val_loss: 0.5362 - val_precision: 0.2857 - val_recall: 0.0564
Epoch 2/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 4s/step - auc: 0.8731 - loss: 0.3577 - precision: 0.4333 - recall: 0.8101 - val_auc: 0.5175 - val_loss: 0.5016 - val_precision: 0.1622 - val_recall: 0.0188
Epoch 3/3
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 4s/step - auc: 0.9619 - loss: 0.2698 - precision: 0.6417 - recall: 0.8087 - val_auc: 0.4956 - val_loss: 0.4810 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - auc: 0.8740 - loss: 0.2534 - precision: 0.6661 - recall: 0.7999

4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - auc: 0.8772 - loss: 0.2538 - precision: 0.6395 - recall: 0.7854 - val_auc: 0.4943 - val_loss: 0.4810 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - auc: 0.9220 - loss: 0.2646 - precision: 0.6196 - recall: 0.7568

4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - auc: 0.9221 - loss: 0.2564 - precision: 0.6188 - recall: 0.7509 - val_auc: 0.4961 - val_loss: 0.4808 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - auc: 0.9404 - loss: 0.2144 - precision: 0.6528 - recall: 0.4347

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - auc: 0.9324 - loss: 0.2206 - precision: 0.6111 - recall: 0.4205 - val_auc: 0.4975 - val_loss: 0.4807 - val_precision: 0.1250 - val_recall: 0.0063
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - auc: 0.9575 - loss: 0.1930 - precision: 0.5919 - recall: 0.6066

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - auc: 0.9562 - loss: 0.1931 - precision: 0.5826 - recall: 0.5944 - val_auc: 0.5003 - val_loss: 0.4807 - val_precision: 0.1000 - val_recall: 0.0063
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - auc: 0.9299 - loss: 0.2009 - precision: 0.6493 - recall: 0.5627

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - auc: 0.9232 - loss: 0.2057 - precision: 0.6306 - recall: 0.5411 - val_auc: 0.5026 - val_loss: 0.4807 - val_precision: 0.0870 - val_recall: 0.0063
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - auc: 0.9445 - loss: 0.2295 - precision: 0.5154 - recall: 0.4657

KeyboardInterrupt: 

In [29]:
finetune_history_df = pd.DataFrame(finetune_histories.history)
history_df.to_csv(RESULTS_DIR + '/finetune_training_history.csv', index=False)

NameError: name 'pd' is not defined

In [ ]:
# efficientnet_model_dir = os.path.join(MODELS_DIR, 'efficientnet_finetune_deepfake_model.h5')
# finetune_model.save(efficientnet_model_dir)

In [ ]:
y_pred = finetune_trained_model.predict(test_gen)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.to_csv(RESULTS_DIR + '/y_pred.csv', index=False)

### Analyze results for finetuned model

In [ ]:
y_pred = pd.read_csv('results/net/y_pred.csv')
y_true = test_gen.labels

In [ ]:
finetune_loaded_history_df = pd.read_csv('results/finetune_training_history.csv')
finetune_loaded_history = {'history': finetune_loaded_history_df.to_dict()}